In [ ]:
%matplotlib inline
import localizer
from localizer import models, keras_helpers, util, visualization
from localizer.localizer import Localizer
from localizer.util import scale_candidates, extract_rois, to_image_coordinates
from localizer.visualization import plot_sample_images, plot_saliency_image
import localizer.config
import matplotlib.pyplot as plt
from scipy.misc import imread
from os.path import join
import os
from pylab import rcParams
import h5py
import keras
import numpy as np
from random import shuffle
import json
import time
rcParams['figure.figsize'] = 15, 15

In [ ]:
beesbook_dir = "/mnt/storage/beesbook"
beesbook_2015 =  join(beesbook_dir, "season_2015_preprocces")
tag_pos_fname = join(beesbook_2015, "tag_positions.json")

with open(join(beesbook_2015, "images.txt")) as f:
    beesbook_images = [l.rstrip('\n') for l in f.readlines()]
    print(beesbook_images[0])
    shuffle(beesbook_images)


In [ ]:
loc = Localizer()

In [ ]:
loc.load_weights("/home/leon/repos/saliency-localizer-models/season_2015")

In [ ]:
loc.compile(image_shape=(4064, 3064))

In [ ]:
def save_json():
    json_obj = {}
    json_obj["time"] = time.time()
    json_obj["directory"] = beesbook_2015
    json_obj["threshold"] = threshold
    json_obj["positions"] = positions
    json_obj["sha1"] = sha1s
    with open(tag_pos_fname, "w+") as f:
        json.dump(json_obj, f)
        
def sha1_of_file(filepath):
    import hashlib
    with open(filepath, 'rb') as f:
        return hashlib.sha1(f.read()).hexdigest()

In [ ]:
progbar = keras.utils.generic_utils.Progbar(len(beesbook_images))
threshold = 0.6
positions = {}
sha1s = {}

try:
    for i, imfname in enumerate(beesbook_images):
        sha1s[imfname] = sha1_of_file(imfname)
        _, candidates, _ = loc.detect_tags(imfname, threshold)
        basename = os.path.basename(imfname)
        positions[basename] = candidates.tolist()
        progbar.add(1)
        break
finally:
    pass
    #save_json()

In [ ]:
def show(imfname, threshold):
    saliencies, candidates, rois = loc.detect_tags(imfname, threshold)
    plt.imshow(visualization.get_roi_overlay(candidates, imread(imfname) / 255.))
    plt.show()
    saliency, image = loc.get_saliency_image(imfname)
    _ = plot_saliency_image(image, saliency, localizer.config.data_imsize)#, figsize=(32, 16))
    plt.show()
    
for threshold in [0.5, 0.6, 0.75, 0.80, 0.85, 0.90, 0.95, 0.995]:
    print(threshold)
    cam1 = "/home/ben/deeplocalizer_data/images/season_2015/cam1/Cam_1_20150911120849_847258_wb.jpeg"
    cam3 = "/home/ben/deeplocalizer_data/images/season_2015/cam3/Cam_3_20150915235539_739596_wb.jpeg"
    show(cam1, threshold)
    #show(cam3, threshold)
    break

In [ ]:
_ = plt.imshow(visualization.get_roi_overlay(candidates, imread(imfile) / 255.))

In [ ]:
with open(tag_pos_fname) as f:
    tag_positions = json.load(f)

In [ ]:
name, candidates = next(iter(tag_positions['positions'].items()))

In [ ]:
_ = plt.imshow(visualization.get_roi_overlay(np.array(candidates), imread(join(beesbook_2015, name)) / 255.))

In [ ]:
hdf5_file = join(beesbook_dir, "season_2015_tags", "tags_plain_64x64.hdf5")
h5 = h5py.File(hdf5_file)
nb_chunks = 1024
roi_shape = (64, 64)
image_dir = join(beesbook_dir, "season_2015_images")
nb_candidates = sum([len(c) for c in tag_positions['positions'].values()])

h5.create_dataset("tags", shape=(nb_candidates, 1, roi_shape[0], roi_shape[1]), dtype='float32',
                  chunks=(nb_chunks, 1, roi_shape[0], roi_shape[1]), compression='gzip')

In [ ]:
h5["tags"].chunks

In [ ]:
h5_pos = 0
def imfname(name):
    return join(image_dir, name.rstrip("_wb.jpeg") + ".jpeg")
def write_to_h5(rois_batch, h5, pos):
    rois = np.concatenate(rois_batch)
    h5_end = pos + len(rois)
    h5["tags"][pos:h5_end] = rois[np.random.permutation(len(rois))]
    h5.flush()
    return h5_end
    
rois_batch = []
progbar = keras.utils.generic_utils.Progbar(len(tag_positions["positions"]))
nb_batches = 96
for i, (name, candidates) in enumerate(tag_positions['positions'].items()):
    try:
        image = imread(imfname(name))
    except OSError as e:
        print(e)
        continue
    rois = extract_rois(candidates, image, roi_shape, border=-32)
    rois_batch.append(rois)
    if len(rois_batch) > nb_batches:
        h5_pos = write_to_h5(rois_batch, h5, h5_pos)
        rois_batch = []
    progbar.update(i)
        
write_to_h5(rois_batch, h5, h5_pos)


In [ ]:
imgs = h5["tags"][:64]
_ = plot_sample_images(imgs, np.zeros((len(imgs), 1)))

In [ ]:
plt.scatter(positions[:n, 0], positions[:n, 1], alpha=0.1)